<a href="https://colab.research.google.com/github/ErickNicolasP/ASE/blob/main/proyecto_ASE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CARGAMOS TODOS LOS DATOS NECESARIOS, BIBLIOTECAS Y ACOMODAMOS LA INFORMACIÓN PARA SU PROXIMA MANIPULACIÓN

In [1]:
#Cargamos el archivo desde google drive
from google.colab import drive
from google.colab import files
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
#importamos las librerías
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d
import io as io

In [3]:
#cargamos los data frame en las variables
#2018
dev_2018 = pd.read_csv('/content/gdrive/MyDrive/Ciencia de Datos/ASE/2018/DEV 2018.csv')
#2019
dev_2019 = pd.read_csv('/content/gdrive/MyDrive/Ciencia de Datos/ASE/2019/DEV 2019.csv')
#2020
dev_2020_enero = pd.read_csv('/content/gdrive/MyDrive/Ciencia de Datos/ASE/2020/DEV 2020_enero-marzo.csv')
dev_2020_julio = pd.read_csv('/content/gdrive/MyDrive/Ciencia de Datos/ASE/2020/DEV 2020_julio-septiembre.csv')
dev_2020_octubre = pd.read_csv('/content/gdrive/MyDrive/Ciencia de Datos/ASE/2020/DEV 2020_octubre-diciembre.csv')
#2022
dev_2022_julio = pd.read_csv('/content/gdrive/MyDrive/Ciencia de Datos/ASE/2022/DEV Julio-Septiembre 2022.csv')
dev_2022_octubre = pd.read_csv('/content/gdrive/MyDrive/Ciencia de Datos/ASE/2022/DEV Octubre-Diciembre 2022.csv')

#poblacion 2020, esta se usará de referencia para sacar la tasa_2018, ya que el censo se hace cada 10 años, el más acertado es el de 2020
censo = pd.read_csv('/content/gdrive/MyDrive/Ciencia de Datos/ASE/censo 2020/censo 2020.csv')

<ipython-input-3-fde523059dcc>:3: DtypeWarning: Columns (496,497) have mixed types. Specify dtype option on import or set low_memory=False.
  dev_2018 = pd.read_csv('/content/gdrive/MyDrive/Ciencia de Datos/ASE/2018/DEV 2018.csv')
<ipython-input-3-fde523059dcc>:5: DtypeWarning: Columns (494,496) have mixed types. Specify dtype option on import or set low_memory=False.
  dev_2019 = pd.read_csv('/content/gdrive/MyDrive/Ciencia de Datos/ASE/2019/DEV 2019.csv')


In [4]:
#Los años 2020 y 2022, no tienen los data frame concatenados, por lo que eso se hará acontinuación
#2020
dev_2020 = pd.concat([dev_2020_enero,dev_2020_julio,dev_2020_octubre], axis=0)
#2022
dev_2022 = pd.concat([dev_2022_julio,dev_2022_octubre],axis=0)

#variables de los data frames
#----dev_2018
#----dev_2019
#----dev_2020
#----dev_2022



#EDAD Y EL SEXO CON SU TASA DE MIGRACIÓN. 2018


In [30]:
hombres_df_2018 = dev_2018.loc[(dev_2018["p1"] == 1) & (dev_2018["p2"] < 97), ["p2", "p1"]].value_counts().reset_index()
hombres_df_2018 = hombres_df_2018.loc[:, ["p2", 0]]
hombres_df_2018.columns = ["edad", "contar"]

mujeres_df_2018 = dev_2018.loc[(dev_2018["p1"] == 2) & (dev_2018["p2"] < 97), ["p2", "p1"]].value_counts().reset_index()
mujeres_df_2018 = mujeres_df_2018.loc[:, ["p2", 0]]
mujeres_df_2018.columns = ["edad", "contar"]

In [31]:
if hombres_df_2018["edad"].max() > mujeres_df_2018["edad"].max() :
    max = hombres_df_2018["edad"].max()
else:
    max = mujeres_df_2018["edad"].max()

In [ ]:
rango_edades = set(range(0, max+1))
edades_presentes = set(hombres_df_2018['edad'])
edades_faltantes = rango_edades - edades_presentes

for edad in edades_faltantes:
    hombres_df_2018.loc[len(hombres_df_2018)] = [edad, 0]
hombres_df_2018 = hombres_df_2018.sort_values("edad").reset_index(drop=True)
hombres_df_2018

In [ ]:
rango_edades = set(range(0, max+1))
edades_presentes = set(mujeres_df_2018['edad'])
edades_faltantes = rango_edades - edades_presentes

for edad in edades_faltantes:
    mujeres_df_2018.loc[len(mujeres_df_2018)] = [edad, 0]
mujeres_df_2018 = mujeres_df_2018.sort_values("edad").reset_index(drop=True)
mujeres_df_2018

In [34]:
tasa_2018 = pd.DataFrame(columns=['hombres', 'mujeres'])
censo = censo.replace(',', '', regex=True).astype('int64')
censo = censo.iloc[0:72]

In [ ]:
tasa_2018["hombres"] = (hombres_df_2018["contar"]/censo["Hombres"])*1000
tasa_2018["mujeres"] = (mujeres_df_2018["contar"]/censo["Mujeres"])*1000
tasa_2018

# EDAD Y SEXO CON SU TASA DE MIGRACIÓN. 2019

In [ ]:
hombres_df_2018 = dev_2018.loc[(dev_2018["p1"] == 1) & (dev_2018["p2"] < 97), ["p2", "p1"]].value_counts().reset_index()
hombres_df_2018 = hombres_df_2018.loc[:, ["p2", 0]]
hombres_df_2018.columns = ["edad", "contar"]

mujeres_df_2018 = dev_2018.loc[(dev_2018["p1"] == 2) & (dev_2018["p2"] < 97), ["p2", "p1"]].value_counts().reset_index()
mujeres_df_2018 = mujeres_df_2018.loc[:, ["p2", 0]]
mujeres_df_2018.columns = ["edad", "contar"]

In [ ]:
if hombres_df_2018["edad"].max() > mujeres_df_2018["edad"].max() :
    max = hombres_df_2018["edad"].max()
else:
    max = mujeres_df_2018["edad"].max()

In [ ]:
rango_edades = set(range(0, max+1))
edades_presentes = set(hombres_df_2018['edad'])
edades_faltantes = rango_edades - edades_presentes

for edad in edades_faltantes:
    hombres_df_2018.loc[len(hombres_df_2018)] = [edad, 0]
hombres_df_2018 = hombres_df_2018.sort_values("edad").reset_index(drop=True)
hombres_df_2018

In [ ]:
rango_edades = set(range(0, max+1))
edades_presentes = set(mujeres_df_2018['edad'])
edades_faltantes = rango_edades - edades_presentes

for edad in edades_faltantes:
    mujeres_df_2018.loc[len(mujeres_df_2018)] = [edad, 0]
mujeres_df_2018 = mujeres_df_2018.sort_values("edad").reset_index(drop=True)
mujeres_df_2018

In [ ]:
tasa_2018 = pd.DataFrame(columns=['hombres', 'mujeres'])
censo = censo.replace(',', '', regex=True).astype('int64')
censo = censo.iloc[0:72]

In [ ]:
tasa_2018["hombres"] = (hombres_df_2018["contar"]/censo["Hombres"])*1000
tasa_2018["mujeres"] = (mujeres_df_2018["contar"]/censo["Mujeres"])*1000
tasa_2018

#EDAD Y SEXO CON SU TASA DE MIGRACIÓN. 2020

In [ ]:
hombres_df_2018 = dev_2018.loc[(dev_2018["p1"] == 1) & (dev_2018["p2"] < 97), ["p2", "p1"]].value_counts().reset_index()
hombres_df_2018 = hombres_df_2018.loc[:, ["p2", 0]]
hombres_df_2018.columns = ["edad", "contar"]

mujeres_df_2018 = dev_2018.loc[(dev_2018["p1"] == 2) & (dev_2018["p2"] < 97), ["p2", "p1"]].value_counts().reset_index()
mujeres_df_2018 = mujeres_df_2018.loc[:, ["p2", 0]]
mujeres_df_2018.columns = ["edad", "contar"]

In [ ]:
if hombres_df_2018["edad"].max() > mujeres_df_2018["edad"].max() :
    max = hombres_df_2018["edad"].max()
else:
    max = mujeres_df_2018["edad"].max()

In [ ]:
rango_edades = set(range(0, max+1))
edades_presentes = set(hombres_df_2018['edad'])
edades_faltantes = rango_edades - edades_presentes

for edad in edades_faltantes:
    hombres_df_2018.loc[len(hombres_df_2018)] = [edad, 0]
hombres_df_2018 = hombres_df_2018.sort_values("edad").reset_index(drop=True)
hombres_df_2018

In [ ]:
rango_edades = set(range(0, max+1))
edades_presentes = set(mujeres_df_2018['edad'])
edades_faltantes = rango_edades - edades_presentes

for edad in edades_faltantes:
    mujeres_df_2018.loc[len(mujeres_df_2018)] = [edad, 0]
mujeres_df_2018 = mujeres_df_2018.sort_values("edad").reset_index(drop=True)
mujeres_df_2018

In [ ]:
tasa_2018 = pd.DataFrame(columns=['hombres', 'mujeres'])
censo = censo.replace(',', '', regex=True).astype('int64')
censo = censo.iloc[0:72]

In [ ]:
tasa_2018["hombres"] = (hombres_df_2018["contar"]/censo["Hombres"])*1000
tasa_2018["mujeres"] = (mujeres_df_2018["contar"]/censo["Mujeres"])*1000
tasa_2018

#EDAD Y SEXO CON SU TASA DE MIGRACIÓN 2022



In [ ]:
hombres_df_2018 = dev_2018.loc[(dev_2018["p1"] == 1) & (dev_2018["p2"] < 97), ["p2", "p1"]].value_counts().reset_index()
hombres_df_2018 = hombres_df_2018.loc[:, ["p2", 0]]
hombres_df_2018.columns = ["edad", "contar"]

mujeres_df_2018 = dev_2018.loc[(dev_2018["p1"] == 2) & (dev_2018["p2"] < 97), ["p2", "p1"]].value_counts().reset_index()
mujeres_df_2018 = mujeres_df_2018.loc[:, ["p2", 0]]
mujeres_df_2018.columns = ["edad", "contar"]

In [ ]:
if hombres_df_2018["edad"].max() > mujeres_df_2018["edad"].max() :
    max = hombres_df_2018["edad"].max()
else:
    max = mujeres_df_2018["edad"].max()

In [ ]:
rango_edades = set(range(0, max+1))
edades_presentes = set(hombres_df_2018['edad'])
edades_faltantes = rango_edades - edades_presentes

for edad in edades_faltantes:
    hombres_df_2018.loc[len(hombres_df_2018)] = [edad, 0]
hombres_df_2018 = hombres_df_2018.sort_values("edad").reset_index(drop=True)
hombres_df_2018

In [ ]:
rango_edades = set(range(0, max+1))
edades_presentes = set(mujeres_df_2018['edad'])
edades_faltantes = rango_edades - edades_presentes

for edad in edades_faltantes:
    mujeres_df_2018.loc[len(mujeres_df_2018)] = [edad, 0]
mujeres_df_2018 = mujeres_df_2018.sort_values("edad").reset_index(drop=True)
mujeres_df_2018

In [ ]:
tasa_2018 = pd.DataFrame(columns=['hombres', 'mujeres'])
censo = censo.replace(',', '', regex=True).astype('int64')
censo = censo.iloc[0:72]

In [ ]:
tasa_2018["hombres"] = (hombres_df_2018["contar"]/censo["Hombres"])*1000
tasa_2018["mujeres"] = (mujeres_df_2018["contar"]/censo["Mujeres"])*1000
tasa_2018